In [1]:
# TODO which one?
#git clone https://github.com/lucidrains/iTransformer.git
#import iTransformer
import sys
sys.path.append('/vol/fob-vol7/nebenf21/reinbene/bene/MA/iTransformer') 
from iTransformer import iTransformer

import torch
import torch.optim as optim
from tqdm import tqdm
from torch.utils.tensorboard import SummaryWriter
import pandas as pd
from pathlib import Path

window_size = 96
pred_length = (96)

from utils import data_handling, training_functions, helpers
import config 

print("Import succesfull")

Import succesfull


# Benchmark all datasets used for transfer learning on iTransformer on its own.

After the sanity check, we will use 96h as the input and the prediction horizon. In the following we will use the RevIn normalization strategie to benchmark iTransformer on our trnasfer learning datasets to get an initial baseline. This baseline will be compared to an SARIMA implementation and an additional DL model.

Those results will also be used to evaluate the transfer-learning capability of iTransformer.

In [2]:
# use electricity dataset
data_dict = data_handling.load_electricity()

electricity = {}
electricity["dataloader_train"], electricity["dataloader_validation"], electricity["dataloader_test"] = data_handling.convert_data(data_dict, window_size, pred_length)


Feature batch shape: torch.Size([32, 96, 348])


In [3]:
# eu electricity data
data_tensor = data_handling.eu_electricity_to_tensor()
data_dict, standardize_values =  data_handling.train_test_split_eu_elec(data_tensor, standardize=True)

# convert to dataloader
europe = {}
europe["dataloader_train"], europe["dataloader_validation"], europe["dataloader_test"] = data_handling.convert_data(data_dict, window_size, pred_length)


Feature batch shape: torch.Size([32, 96, 208])


In [4]:
# bavaria dataset
data_tensor = data_handling.load_bavaria_electricity()
data_dict, standadizer = data_handling.train_test_split_eu_elec(data_tensor, standardize=True)

# convert to datalaoder
bavaria = {}
bavaria["dataloader_train"], bavaria["dataloader_validation"], bavaria["dataloader_test"] = data_handling.convert_data(data_dict, window_size, pred_length)

Feature batch shape: torch.Size([32, 96, 67])


In [5]:
datasets = {"electricity" : electricity,
			"europe" : europe,
			"bavaria" : bavaria
            }

In [12]:
# run experiment for each normalizaiton strategie and save model and evaluation metrics

for key, dataset_dict in datasets.items():
    torch.cuda.empty_cache()
    inputs, _ = next(iter(dataset_dict["dataloader_train"]))
    num_variates = inputs.size(2)
    
    # define parameters and create config 
    best_parameters = {'depth': 2, 'dim': 256, 'dim_head': 56, 'heads': 4, 'attn_dropout': 0.2, 'ff_mult': 4, 'ff_dropout': 0.1, 
                    'num_mem_tokens': 4, 'learning_rate': 0.0005}


    model_config = {
        'num_variates': num_variates,
        'lookback_len': window_size,
        'depth': best_parameters["depth"],
        'dim': best_parameters["dim"],
        'num_tokens_per_variate': 1,
        'pred_length': pred_length,
        'dim_head': best_parameters["dim_head"],
        'heads': best_parameters["heads"],
        'attn_dropout': best_parameters["attn_dropout"],
        'ff_mult': best_parameters["ff_mult"],
        'ff_dropout': best_parameters["ff_dropout"],
        'num_mem_tokens': best_parameters["num_mem_tokens"],
        'use_reversible_instance_norm': True,
        'reversible_instance_norm_affine': True,
        'flash_attn': True
    }

    # select available deviec
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    print(f"Using device: {device}")

    # defining all needed instances
    model = iTransformer(**model_config).to(device)
    optimizer = optim.Adam(model.parameters(), lr=best_parameters["learning_rate"])
    scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=10, gamma=0.1)
    writer = SummaryWriter(log_dir=config.CONFIG_LOGS_PATH["iTransformer_baseline"])

    # run model training as mentioned in the original paper
    epoch = 15

    for epoch in range(1, epoch + 1):
        training_functions.train_one_epoch(epoch, model, device, dataset_dict["dataloader_train"], dataset_dict["dataloader_validation"], optimizer, scheduler, writer)


    metrics = helpers.full_eval(model, dataset_dict["dataloader_test"], device)

    for eval_metric, value in metrics[96].items():
        metrics[96][eval_metric] = value.item()

    print(metrics)

    metrics_df = pd.DataFrame.from_dict(metrics[96], orient='index')
    metrics_df.rename(columns={0: key}, inplace=True)

    metrics_df.to_csv(f"{config.CONFIG_OUTPUT_PATH['iTransformer_baseline']}/metrics_{key}_epochs{epoch}_revin.csv")

Using device: cuda


Epoch: 1: 100%|██████████| 151/151 [00:06<00:00, 24.57it/s]


Epoch 1, MSE-Loss: 0.06541388553400702, LR: 0.0005


Epoch: Validating: 100%|██████████| 21/21 [00:00<00:00, 84.29it/s]


Validation MAE is {96: {'mse': tensor(0.2419, device='cuda:0')}, 192: {'mse': 0}, 336: {'mse': 0}, 720: {'mse': 0}}


...

Epoch 15, MSE-Loss: 5.64124156674304e-05, LR: 5e-05
Checkpointing succesfull after epoch 15


Epoch: Validating: 100%|██████████| 33/33 [00:00<00:00, 148.71it/s]


Validation MAE is {96: {'mse': tensor(0.0002, device='cuda:0')}, 192: {'mse': 0}, 336: {'mse': 0}, 720: {'mse': 0}}


Epoch: Validating: 100%|██████████| 71/71 [00:00<00:00, 116.37it/s]


{96: {'mse': 0.0003162012726534158, 'mae': 0.00982731394469738, 'p10': 0.0011721873888745904, 'p50': 0.0066027529537677765, 'p90': 0.019870752468705177}, 192: {'mse': 0, 'mae': 0, 'p10': 0, 'p50': 0, 'p90': 0}, 336: {'mse': 0, 'mae': 0, 'p10': 0, 'p50': 0, 'p90': 0}, 720: {'mse': 0, 'mae': 0, 'p10': 0, 'p50': 0, 'p90': 0}}
